In [ ]:
#Import required packages
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import pandas as pd
from Helper_functions import load_dict

from AI_functions import resnet18,data_generator

from AI_functions import resnet18, BarlowTwins, ImageCollateFunction,BarlowTwinsLoss,CellDataset,data_generator

latent_dim=512
epochs=10

In [ ]:
# custom dataset
class CellDataset():
    def __init__(self, images,labels,ID, transforms=None):
        self.X = images
        self.Y=  labels
        self.Z= ID
        self.transforms = transforms
         
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        data = self.X[i]
        label=self.Y[i]
        ID=self.Z[i]
        data = np.asarray(data).astype(np.uint8)

        if self.transforms:
            data1 = self.transforms(data)
            data2 = self.transforms(data)
        
        return (data1,data2),np.array((label,ID))


In [ ]:
DATASETS=["Retina_1_2","Colon","Choroid","BoneMarrow_sample1"]

train_data_list=[]
test_data_list=[]

cutoff=30000


for N,DATASET in enumerate(DATASETS):
    basepath = r"C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Data\\" +DATASET
    outpath = basepath + "\\Outputs"
    image_dim=64 #Dim of the final images
    nuclear_channel="Ch7"
    cellmask_channel="Ch1_mask"
    df=pd.read_csv(outpath+"\\cell_info.csv")
    cell_names=df["Cell_ID"].to_numpy()
    Prediction_Channels=None

    image_dict=load_dict(outpath,cell_names,image_dim)

    Channels=['Ch1']  #Channel to be fed to the NN
    images_with_index = []

    for image_i in image_dict:
        #print(image_dict[image_i].keys())
        if len(image_dict[image_i].keys())>=len(Channels):
            image=cv.merge([image_dict[image_i][i] for i in Channels])
            images_with_index.append((int(image_i),image))
        else:
            print(image_i)
        
    images=np.array([image[1] for image in images_with_index])
    names=np.array([image[0] for image in images_with_index])
    assert sum(names!=df['Cell_ID'].to_numpy()) ==0  #Check that the order has been preserved
    DNA_pos=df["DNA_pos"].to_numpy()
    Touches_Boundary=df["Touches_boundary"].to_numpy()
    labels=np.array([N for i in names]) #Labels are the tissues

    Thresh=50
    plt.hist(df["Gradient RMS_M01_Ch01"],bins=200);
    plt.axvline(x=Thresh,color="red")
    plt.show()
    idx_to_keep=np.array(Touches_Boundary==0,dtype=int)+np.array(df["Gradient RMS_M01_Ch01"]>Thresh,dtype=int)==2 #np.array(DNA_pos==1,dtype=int)+
    #Filter
    print(len(images))
    images=images[idx_to_keep]
    names=names[idx_to_keep]
    labels=labels[idx_to_keep]

    images=images[:cutoff]
    names=names[:cutoff]
    labels=labels[:cutoff]


    mini=int(round(abs(np.array(images).min()),0))
    images=images+abs(np.array(images).min())
    mean=np.array(images).mean()
    maxi=np.array(images).max()
    std=np.array(images).std()

    print(len(images))


    returned=data_generator(images,labels,names,mini,train_test_split = 0.8,batch_size = 100,sample=False)
    train,test,batch_size,mean_loader,std_loader=returned
    [train_data,train_data1,train_labels,train_ID]=train
    [test_data,test_data1,test_labels,test_ID]=test




    rotated = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomRotation(degrees=18,fill=mini),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean_loader], std=[std_loader]),  # for grayscale images
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.Lambda(lambda x: x if 0.5>np.random.rand() else transforms.functional.invert(x)-1),
        transforms.Lambda(lambda x: x *(1+0.2*np.random.randn()) ),
    # transforms.Lambda(lambda x: x + (0.1**0.5)*torch.randn(64, 64) )
        ]) 

    transform_test = transforms.Compose(
        [transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[mean_loader], std=[std_loader])
         ])



    train_data = CellDataset(train_data1,train_labels,train_ID, rotated)
    train_data_list.append(train_data)

    test_data = CellDataset(test_data1,test_labels,test_ID, transforms=transform_test)
    test_data_list.append(test_data)



from torch.utils.data import ConcatDataset
train_cat = ConcatDataset([train_data for train_data in train_data_list])
train_cat_loader = DataLoader(train_cat, batch_size=524,shuffle=True,drop_last=True)


test_cat = ConcatDataset([test_data for test_data in test_data_list])
test_cat_loader = DataLoader(test_cat, batch_size=524,shuffle=True)


    

In [ ]:
#import dill

#torch.save(train_cat_loader,'train_data.pth',pickle_module=dill)

In [ ]:
import sys
sys.getsizeof(images)

In [ ]:
my_imgs=next(iter(train_cat_loader))
plt.imshow(my_imgs[0][0][2][0],vmin=-10,vmax=10)
plt.colorbar()

plt.show()
plt.imshow(my_imgs[0][1][2][0],vmin=-10,vmax=10)
plt.colorbar()
plt.show()
my_imgs=next(iter(test_cat_loader))
plt.imshow(my_imgs[0][0][2][0],vmin=-10,vmax=10)
plt.colorbar()
plt.show()

plt.imshow(my_imgs[0][1][2][0],vmin=-10,vmax=10)
plt.colorbar()
plt.show()

In [ ]:
Train_net=torch.load(r'C:\Users\Thibaut Goldsborough\Documents\Seth_BoneMarrow\Results\\'+"model300loss_tensor(93.1900).pth")


In [ ]:
train_loader = train_cat_loader
test_loader = test_cat_loader

def get_val():
    total_loss = 0
    model.eval()
    for (x0, x1), _ in test_loader:
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model(x0)
        z1 = model(x1)
        loss = criterion(z0, z1)
        total_loss += loss.detach()

    print("Validation Loss:",int(total_loss / len(test_loader)))
    return int(total_loss / len(test_loader))
    
def train():
    total_loss = 0
    model.train()
    for (x0, x1), _ in train_loader:
        x0 = x0.to(device)
        x1 = x1.to(device)
        z0 = model(x0)
        z1 = model(x1)
        loss = criterion(z0, z1)
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_loss = total_loss / len(train_loader)
    return avg_loss


#resnet = resnet18(num_classes=latent_dim,channel_num=len(Channels))
#backbone = resnet
#model = BarlowTwins(backbone,latent_dim)

model=Train_net

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

collate_fn = ImageCollateFunction(input_size=32)

criterion = BarlowTwinsLoss(device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.06)

print("Starting Training")

train_losses=[]
val_losses=[]
for epoch in range(epochs):
    avg_loss=train()
    print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")
    train_losses.append(avg_loss)
    val_losses.append(get_val())

In [ ]:
train_losses=[int(i.detach().cpu()) for i in train_losses]
plt.figure(figsize=(10,10))
plt.plot(train_losses)
plt.plot(val_losses)
plt.savefig(basepath+"/Results/Loss"+str(train_losses[-1])+".png",bbox_inches="tight")


In [ ]:
train_loader = DataLoader(train_cat, batch_size=batch_size,shuffle=True,drop_last=True)

In [ ]:
#train_data = CellDataset(train_data1,train_labels,train_ID,rotated)
#train_loader = DataLoader(train_data, batch_size=124, shuffle=True)#,sampler=sampler)


latent_dim=2048


barlow=np.zeros((len(train_loader)*batch_size,latent_dim))
names=np.zeros((len(train_loader)*batch_size))
labels=np.zeros((len(train_loader)*batch_size))
i=0
for (x0, x1), Y in train_loader:
    x0 = x0.to(device)
    x1 = x1.to(device)
    latents=model(x0).detach().cpu().numpy()  #model.backbone(x0).detach().cpu().numpy()
    names[i:i+len(latents)]=Y[:,1]
    labels[i:i+len(latents)]=Y[:,0]
    barlow[i:i+len(latents)]=latents
    i+=len(latents)
    print(i,len(train_data1))

df3=pd.DataFrame(barlow)
df3["Cell_ID"]=names
df3["Dataset"]=[DATASETS[int(i)] for i in labels]
#df3.to_csv(basepath+"/Results/Barlow_latents"+str(train_losses[-1])+".csv", index = False, header=True)

    

In [ ]:
df=df3
cell_names=df["Cell_ID"].to_numpy()
Labels=df["Dataset"].to_numpy()

latents=df.iloc[:,:].drop(["Cell_ID","Dataset"],axis=1).to_numpy()
print(latents.shape)

In [ ]:
COLOR_DICT={"BoneMarrow_sample1":"red","Retina_1_2":"limegreen","Colon":"magenta","Choroid":"blue"}
inv_map = {v: k for k, v in COLOR_DICT.items()}
colors=[COLOR_DICT[i] for i in Labels]

In [ ]:
from sklearn.manifold import TSNE
fit = TSNE(n_components=2)
u = fit.fit_transform(latents)


In [ ]:
import matplotlib.patches as mpatches

fig=plt.figure(figsize=(15,10))

patches=[mpatches.Patch(color=color, label=tissue) for tissue,color in COLOR_DICT.items()]
plt.legend(handles=patches)
plt.scatter(u[:,0],u[:,1],c=colors,s=1)

In [ ]:
df["U0"]=u[:,0]
df["U1"]=u[:,1]
df.to_csv(basepath+df_path)